In [47]:
import platform
op_sys = platform.system()
if op_sys == 'Windows':
    dtdir = 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\'
    
elif op_sys == 'Linux':
    dtdir = '/home/cihat/Dropbox/code/special/data/'
import os
os.chdir(dtdir)
import glob
from import_tm import *
# go to output dir
dtdir = 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\'
os.chdir(dtdir)
from par_import import *
# from newpm_import import *
import pandas as pd
import numpy as np
#%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
from numpy.polynomial import Legendre as L


In [48]:
dtdir = 'C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\data\\output\\'
import os
os.chdir(dtdir)

In [49]:
def linearize(data, sec_lin = False):
    # This function linearize the m0. The linerization functions has to be modified each time.
    # For Forward Scattering Excluding GE256GE063 which uses defaults
    k1 = 1; l1 = 0.6; k2 = 3.4; l2 = 1.78
    e = 1
    data['Rf'] = np.log((abs(((k1*data.nr + np.log(k2*data.ni)*1j )**2 -1)/((l1*data.nr + np.log10(l2*data.ni)*1j)**2 + 2)))**e)
    # Defaults
#    k1 = 1.20; l1 = 0.66; k2 = 4.3; l2 = 1
#    e = 2 # Better Multiple Regression comparing to e = 1
#    data['Rf'] = np.log((abs(((k1*data.nr + np.log10(k2*data.ni)*1j )**2 -1)/((l1*data.nr + np.log10(l2*data.ni)*1j)**2 + 2)))**e)



    if sec_lin:
        # this needs to be modified. It is the best fit of LPi against m0. fit function obtained from matlab. Needs to be modified for each LPi
        # Incase of LP4 both linear sinisoidal function and 1st degre pol. regression fit well. Thus 2nd linearization may not be necessary
        data[m] = (-1.605e-09)*(np.sin(data.m0/2 -0.7*np.pi))+2.237e-09*((data.m0 + 1))
    return data

def Leg_Pol (x,deg):

    """
    deg is the degree of legendre pol. 3 means Legendre polynomials include
    Po + P1 + P2 + P3

    x is the independent variable in the Leg. Pol
    """

    P0 = 1
    P1 = x
    lps = [P0, P1]

    for n in range(1, deg):
        p = 'P{}'.format(n)
        p_next = 'P{}'.format(n+1)
        p_prev = 'P{}'.format(n-1)
        vars()[p_next] = ((2*n + 1)*x*vars()[p] - n*vars()[p_prev])/(n+1)
        lps.append(vars()[p_next])

        return lps

In [50]:
variables = pd.read_csv(dtdir+'output_list.csv',dtype={'N':int,'xm':str,'nr':str,'ni':str})
variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   N       343 non-null    int32 
 1   xm      343 non-null    object
 2   nr      343 non-null    object
 3   ni      343 non-null    object
dtypes: int32(1), object(3)
memory usage: 9.5+ KB


In [51]:
variables['x_m'] = variables['xm'].str.replace('.','', regex=False)
variables.loc[:,'xm'] = variables['xm'].apply(float)
variables['n_r'] = variables['nr'].str.replace('.','', regex=False)
variables.loc[:,'nr'] = variables['nr'].apply(float)
variables['n_i'] = variables['ni'].str.replace('.','', regex=False)
variables.loc[:,'ni'] = variables['ni'].apply(float)
variables

,N,xm,nr,ni,x_m,n_r,n_i
0,1024,0.336,1.30,0.001,03360,130,00010
1,1024,0.336,1.30,0.004,03360,130,00040
2,1024,0.336,1.30,0.023,03360,130,00230
3,1024,0.336,1.30,0.045,03360,130,00450
4,1024,0.336,1.30,0.090,03360,130,00900
...,...,...,...,...,...,...,...
338,64,0.730,1.68,0.023,07300,168,00230
339,64,0.832,1.68,0.023,08320,168,00230
340,64,0.832,1.71,0.045,08320,171,00450
341,64,0.960,1.68,0.023,09600,168,00230


In [66]:
s = 0
alpha_value = 1.0
_log = []
theta = np.arange(0, 181)

# Setting boundaries for angular fit
b1 = 0
b2 = 7
b_end = 181

# Setting the b2 to fit forward scattering (where b1=0)
# if b1 == 0:
#    if N <= 256:
#        b2 = 20
#    elif N == 512:
#        b2 = 13
#    elif N == 1024:
#        b2 = 6
# else:
#    b2 = b2
#b2 = 7
# Set te degree of Legendre Polynomials
dg = 2

# Setting angular boundaries for the plots
theta = np.arange(0, 181)
a1 = 0
a2 = 181
x1 = theta[a1:a2]

dt = []  # to be used to write Legend. Pol. coefficients data


for i in variables.index:
    N=variables.loc[i,'N']
    xm = variables.loc[i,'xm']
    nr = variables.loc[i,'nr']
    ni = variables.loc[i, 'ni']
    x_m = variables.loc[i,'x_m']
    n_r = variables.loc[i,'n_r']
    n_i = variables.loc[i,'n_i']
    runid = '{}_{}_{}_{}'.format(N, x_m, n_r, n_i)
    try:
        missing = 't-matrix'
        tm_eff = vars()['tm'+runid+'_eff']
        missing = ' f-fasm'
        pm_eff = vars()['newpm'+runid+'_eff']
        s += 1

    except KeyError:
        #print('missing run {} {}_{}_{}_{}_eff'.format(missing, N, xm, nr, ni))
        _log.append('missing run {} {}_{}_{}_{}_eff'.format(
            missing, N, xm, nr, ni))
        continue

    tm_eff['cs'] = tm_eff.qs[0]*np.pi*(xm*N**(1/3))**2
    tm =  vars()['tm'+runid+'_mx']
    pm = vars()['newpm'+runid+'_mx']
    old_pm = vars()['param'+runid+'_mx']

    #Some parameters to use later
    s = 0
    alpha_value = 1.0
    _log = []
    #Setting b2 manually
    #b2=13
    b1=0
    b2=180

    x = theta[b1:b2]

    new_p22_err = (tm.P22*tm.P11 - pm.P22)[b1:b2]
    old_p22_err = (tm.P22*tm.P11 - old_pm.P22)[b1:b2]
    maxpol_tm = tm.theta[-tm.P12 == max(-tm.P12)].values[0]
    maxpol_pm = pm.theta[-pm.P12/pm.P11 == max(-pm.P12/pm.P11)].values[0]

    # Now the correction
    # First fit the error function with a 3rd Degree Legendre Polynomial
    xw = 2*(x-x[0])/(x[-1]-x[0]) - 1

    coeffs = L.fit(x, old_p22_err, deg=dg) # coeffs are mapped to [-1,1] window since L.P. are orthogonal in this domain
    coeff = coeffs.convert() # this converts the coeffs in to the original domain. These are the coeffs that will be analyzed statistically and passed to New Parameterization model
    plotting = False
    if plotting:
#                if mod(s,7) == 0:
        # Now Let's Correct the parametrization and plot the new and old comparison with tmatrix
#                    p22_corr = pd.Series(np.polynomial.legendre.legval(x, coeffs), index=x)
#                    new_pm = pm.copy()
#                    new_pm.P22 = pm.P22.add(p22_corr, fill_value=0)
        fig, ax = plt.subplots(ncols=2)
        p22_err = (tm.P11 - old_pm.P11)[a1:a2]
        ax[0].plot(((-old_pm.P12/old_pm.P11)), '.', label = 'Old pm')
        ax[0].plot((-tm.P12)[a1:a2], '.', label = 'tm')
        ax[0].set_title('Old P22'+runid)
#                    ax[0].set_yscale('log')
        ax[0].legend()

        # new fit
        ax[1].plot((-pm.P12/pm.P11), '.', label = 'new_pm')
        ax[1].plot((-tm.P12), '.', label = 'tm')
        ax[1].legend()
        ax[1].set_title('New P22'+runid)

        # Error Function and the Fit
#                    ax[2].plot(x, new_p22_err, '.', label = 'new_p22_err')
        #ax[2].plot(x, old_p22_err,'.', label = 'old_p22_err' )
        #ax[2].legend()
#                    ax[2].set_title('p22 error & the fit')

        # old and new p22
#                    ax[3].plot((old_pm.P22), '.', label = 'old_pm')
#                    ax[3].plot((pm.P22), '.', label = 'new_pm')
#                    ax[3].set_title('Old & New P22'+runid)
#                    ax[3].legend()
        plt.grid()
        plt.tight_layout()
        plt.show()
        plt.close()

                    # Calculating Param Csca from Qs
    aprj=((np.pi * xm**2) * (N**(2/3)))
    c_sca_m_1 = 0.200
    c_sca_m_2 = 0.164
    c_sca_m_3 = 1.047
    c_sca_m_4 = 0.127
    c_sca_x_1 = 3.082
    c_sca_x_2 = 0.757
    m = complex(nr, ni)
    m0 = abs((m**2-1)/(m**2+2))
    corr_sca = 1. + c_sca_m_3 * (m0-c_sca_m_4)* np.sin(c_sca_x_1 * xm) * np.exp(-xm * c_sca_x_2)
    sw={}

    sw['N'] = N
    sw['xm'] = xm
    sw['nr'] = nr
    sw['ni'] = ni
    sw['taus_out'] = pm_eff.taus[0]
    sw['taua_out'] = pm_eff.taua[0]
#                sw['qa_tm'] = tm_eff.qa[0]
    sw['qs_tm'] = tm_eff.qs[0]
#                sw['qa_pm'] = pm_eff.qa[0]
    sw['qs_pm'] = pm_eff.qs[0]
#                sw['cs_pm'] = pm_eff.cs[0]
#                sw['cs_tm'] = tm_eff.cs[0]
#                sw['Aproj'] = aprj
#                sw['corr_sca'] = corr_sca
    sw['m0'] = m0
#                sw['maxpol_tm'] = maxpol_tm
#                sw['maxpol_pm'] = maxpol_pm
#                sw['p1'] = coeff.coef[0]
#                sw['p2'] = coeff.coef[1]
#                sw['p3'] = coeff.coef[2]
#                if dg == 3:
#                    sw['p4'] = coeff.coef[3]
    dt.append(sw)

# Should you need the plots, edit the "if" below

    plotting = False

    if plotting:
    #if np.mod(s,7) == 0:
        # Now Let's Correct the parametrization and plot the new and old comparison with tmatrix
        p22_corr = pd.Series(coeffs(x), index=x)

        new_pm = old_pm.copy()
        new_pm.P22 = (old_pm.P22).add(p22_corr, fill_value=0) # P22_corr does not cover the whole domain, so fill_value is necessary
        fig, ax = plt.subplots(ncols=3)

        ax[0].plot(np.log10((new_pm.P22))[a1:a2], label = 'new_pm python')
        ax[0].plot( (np.log10(tm.P22*tm.P11))[a1:a2] , '.', label = 'tm')
        ax[0].set_title('New P22 Python Correction'+runid)
        ax[0].legend()
        plt.grid()

        # Old fit
        ax[1].plot( (np.log10(old_pm.P22))[a1:a2], label = 'old_pm')
        ax[1].plot( (np.log10(tm.P22*tm.P11))[a1:a2], '.', label = 'tm')
        ax[1].legend()
        ax[1].set_title('Old P22'+runid)
        plt.grid()

        # Error Function and the Fit
#                    ax[2].plot(x, old_p22_err , '.', label = 'p22_err')
#                    ax[2].plot(x, p22_corr,'.', label = 'p22_corr' )
#                    ax[2].legend()
#                    ax[2].set_title('p22 error & the fit')
#                    plt.grid()
#                    plt.tight_layout()
#                    plt.show()
        ax[2].plot(x, old_p22_err , '.', label = 'p22_err')
        ax[2].plot(x, p22_corr,'.', label = 'p22_corr' )
        ax[2].legend()
        ax[2].set_title('P22 error & the fit')
        plt.grid()
        plt.tight_layout()
        plt.show()

lp=1
if lp == 1:
    p = "p1"
elif lp == 2:
    p = 'p2'
elif lp == 3:
    p = 'p3'
elif lp == 4:
    p = 'p4'
else:
    print('wrong p values')


LP_Coeff = pd.DataFrame(dt)
linearize(LP_Coeff)
# optional parameters which makes filtering easier visually.
LP_Coeff['xm_m0'] = LP_Coeff['xm']*LP_Coeff['m0']
LP_Coeff['N_m0'] = LP_Coeff['N']*LP_Coeff['m0']*0.05
LP_Coeff['N_xm'] = LP_Coeff['N']*LP_Coeff['xm']

# Breaking in to 4 regimes
LP512_lt063 = LP_Coeff[(LP_Coeff.N <= 512) & (LP_Coeff.xm < 0.63)]
LP512_gt063 = LP_Coeff[(LP_Coeff.N <= 512) & (LP_Coeff.xm >= 0.63)]
LP1024_lt063 = LP_Coeff[(LP_Coeff.N >= 512) & (LP_Coeff.xm < 0.63)]
LP1024_gt063 = LP_Coeff[(LP_Coeff.N >= 512) & (LP_Coeff.xm >= 0.63)]
LP1024 = LP_Coeff[ LP_Coeff.N >= 512 ]
LP_lt063 = LP_Coeff[LP_Coeff.xm < 0.63]
LP_gt063 = LP_Coeff[LP_Coeff.xm >= 0.63]



In [67]:
LP_Coeff

,N,xm,nr,ni,taus_out,taua_out,qs_tm,qs_pm,m0,Rf,xm_m0,N_m0,N_xm
0,1024,0.336,1.30,0.001,0.000843,0.000574,0.245158,0.189306,0.186993,1.672859,0.062830,9.574038,344.064
1,1024,0.336,1.30,0.004,0.000844,0.002296,0.244536,0.190233,0.187009,1.682642,0.062835,9.574853,344.064
2,1024,0.336,1.30,0.023,0.000848,0.013196,0.242294,0.197810,0.187552,1.356002,0.063018,9.602675,344.064
3,1024,0.336,1.30,0.045,0.000862,0.025804,0.242021,0.210323,0.189129,0.935703,0.063547,9.683390,344.064
4,1024,0.336,1.30,0.090,0.000919,0.051565,0.249216,0.246082,0.195410,0.303425,0.065658,10.005003,344.064
...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,64,0.730,1.68,0.023,0.049359,0.017317,2.114412,2.202473,0.378126,1.178237,0.276032,1.210003,46.720
267,64,0.832,1.68,0.023,0.082770,0.021213,2.850684,3.180203,0.378126,1.178237,0.314601,1.210003,53.248
268,64,0.832,1.71,0.045,0.089093,0.041006,2.940252,3.325695,0.391559,0.820393,0.325777,1.252990,53.248
269,64,0.960,1.68,0.023,0.142400,0.026758,3.806748,4.587454,0.378126,1.178237,0.363001,1.210003,61.440
